In [113]:
import csv
from sklearn import tree
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [114]:
def findCommunities():
    communities = []
    with open('C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv', 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities

In [134]:
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain = xPositive[:indxP] + xNegative[:indxN]
        yTrain = yPositive[:indxP] + yNegative[:indxN]
        xTest = xPositive[indxP:] + xNegative[indxN:]
        yTest = yPositive[indxP:] + yNegative[indxN:]
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features


In [135]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

In [136]:
calcualtePercentage(Y)

(0.62719518314099343, 0.37280481685900652)

Explaination
1a) The above code snippet is used to find if ViolentCrimesPerPop is greater than 0.1 or not. If it is greater than 0.1, then highCrime/y is appended with 1 (1 means true) and if it less than 0.1, then highCrime/y is appended with 0 ( 0 means false). The positive and negative instances are 62.71 % and 37.29% respectively. 

In [137]:
"""def randomForest():
    return RandomForestClassifier.RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, 
                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                       max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, 
                                       n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)"""

"def randomForest():\n    return RandomForestClassifier.RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2, \n                                       min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', \n                                       max_leaf_nodes=None, min_impurity_split=1e-07, bootstrap=True, oob_score=False, \n                                       n_jobs=1, random_state=None, verbose=0, warm_start=False, class_weight=None)"

In [138]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

In [139]:
def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)

In [140]:
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)

In [141]:
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres

In [142]:
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec

In [143]:
def find_fmeasure(prec,rec):
    tmp = []
    for i,j in zip(prec,rec):
        tmp.append(2.0*(i*j)/(i+j))
    return tmp

In [144]:
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)
    f_score =find_fmeasure(precision,recall)
    print ("F_score", f_score)

In [145]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        evaluation(y[test_ind],predictions)
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg

In [146]:
distintCommunities = findCommunities()

In [147]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
XTrain,YTrain,XTest,YTest,Features = extractData(filename,distintCommunities,addstate = False,addcommunity = False)

In [148]:

rf = RandomForestClassifier(max_depth=None, min_samples_split=2,n_estimators=10)
rf.fit(XTrain,YTrain)
predictedY=rf.predict(XTest)


In [149]:
confusionMatrix(YTest,predictedY)

array([[401,  99],
       [ 70, 228]])

 Explaination :
 The confusion Matrix for Y( i.e HighCrime ) can be found above.

In [150]:
evaluation(YTest,predictedY)

Confusion Matrix
[[401  99]
 [ 70 228]]
Accuracy 0.788220551378
Precision [0.85138004246284504, 0.69724770642201839]
Recall [0.80200000000000005, 0.7651006711409396]
F_score [0.82595262615859932, 0.72960000000000003]


In [151]:
do_cross_validation(X[:1000],Y[:1000],rf,10)

Confusion Matrix
[[64 10]
 [14 12]]
Accuracy 0.76
Precision [0.82051282051282048, 0.54545454545454541]
Recall [0.86486486486486491, 0.46153846153846156]
F_score [0.84210526315789469, 0.49999999999999989]
Confusion Matrix
[[89  2]
 [ 5  4]]
Accuracy 0.93
Precision [0.94680851063829785, 0.66666666666666663]
Recall [0.97802197802197799, 0.44444444444444442]
F_score [0.96216216216216222, 0.53333333333333333]
Confusion Matrix
[[87  3]
 [ 3  7]]
Accuracy 0.94
Precision [0.96666666666666667, 0.69999999999999996]
Recall [0.96666666666666667, 0.69999999999999996]
F_score [0.96666666666666667, 0.69999999999999996]
Confusion Matrix
[[78  4]
 [ 9  9]]
Accuracy 0.87
Precision [0.89655172413793105, 0.69230769230769229]
Recall [0.95121951219512191, 0.5]
F_score [0.92307692307692313, 0.58064516129032262]
Confusion Matrix
[[48  4]
 [ 9 39]]
Accuracy 0.87
Precision [0.84210526315789469, 0.90697674418604646]
Recall [0.92307692307692313, 0.8125]
F_score [0.88073394495412849, 0.85714285714285721]
Confusion

0.81099999999999994

Explaination: Cross Validation
1-c-i) The 10 cross validation's accuracy, Precision, Recall and F_score can be found above.

1-c-ii) 
    

In [152]:
sumCorrect =0
for i in range(1000,len(Y)):
    if Y[i]==predictedY[i-1000]:
        sumCorrect += 1
sumCorrect*1.0/len(Y[1000:])      


IndexError: index 798 is out of bounds for axis 0 with size 798

In [85]:
featureImportance = rf.feature_importances_


In [86]:
featureImp = []
for i in range(len(Features)):
    if featureImportance[i]>0:
        featureImp.append((Features[i],featureImportance[i]))


In [87]:
sorted(featureImp, key = lambda x:x[1],reverse= True)

[('PctPersDenseHous', 0.079366757950437988),
 ('PctPopUnderPov', 0.057195434470095952),
 ('racePctWhite', 0.055718311205104662),
 ('PctKids2Par', 0.048743629936113114),
 ('PctYoungKids2Par', 0.043700134906256023),
 ('PctIlleg', 0.04337644588181435),
 ('PctTeen2Par', 0.024284495477983479),
 ('PctFam2Par', 0.022425793698365497),
 ('racepctblack', 0.020111144179171966),
 ('PopDens', 0.019662311038943062),
 ('perCapInc', 0.016950856636976673),
 ('NumUnderPov', 0.014308927672348671),
 ('TotalPctDiv', 0.014266420081045316),
 ('pctWInvInc', 0.014165307745639871),
 ('HousVacant', 0.013734714299064999),
 ('MedRent', 0.013182306500764417),
 ('PctImmigRec10', 0.01236784416691338),
 ('PctSpeakEnglOnly', 0.012238475799794733),
 ('PctBornSameState', 0.011693340860639915),
 ('HispPerCap', 0.011254208816908003),
 ('PctNotSpeakEnglWell', 0.011216729534474621),
 ('PctOccupManu', 0.011084447910859234),
 ('PctEmplProfServ', 0.010993932433052885),
 ('FemalePctDiv', 0.010571491086046176),
 ('PctUnemployed',

Explaination : Decision Tree Classifier
1-b-ii ) The top features for decision tree can be found above. The reason that these are the best features is because at each split point, decision tree will choose the feature which "best" splits the observations. What qualifies as best varies, but generally the split is done so that the subsequent nodes are more homogenous/pure with respect to the target. There are different ways of measuring homogeneity, for example Gini, Entropy, Chi-square. 
Considering the accuracy of this alogorithm, I think new features can be added/removed to increase the accuracy of the algorithm.